# Train YOLOv5 (and v8 soon!) on Weed-AI Datasets

This guide will take you through training a state-of-the-art object detection architecture - YOLOv5 - on Weed-AI datasets. It combines elements of the official Ultralytics guide, with elements of other custom training and conversion guides.

**Steps:**
1. Setup the project: creating folders, cloning YOLOv5
2. Download the Weed-AI dataset
3. Convert weedCOCO to YOLO annotation format
4. Create YOLOv5 supporting files
5. Train YOLOv5
6. Inference on pictures/videos

The tutorial requires you to have access to a Google Drive account and be able to upload images/data to specific folders. Algorithms will train fastest with a GPU. Select the GPU type under 'Runtime' > 'Change Runtime Type'. Make sure GPU is selected. Premium or High RAM will improve speed/size of models that can be trained.
Make sure you run each cell in the tutorial by pressing the 'Play' button on the left hand side. Some options that may need changing are in capital letters.



In [2]:
# mount google drive - this gives the Colab notebook access to your Drive. It may ask you for permission/to sign in too.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create Project Folder

To begin, create a project folder in your Google Drive. We'll call this one `weedai_yolo`. Replace this with whatever name you decide.

It will be created in the root folder of your Google Drive. InTO this folder we'll be cloning the [YOLOv5 GitHub Repository](https://github.com/ultralytics/yolov5) and saving our data too. There are many guides on training YOLOv5 that are accessible through the official repository, make sure to check those for any tips/tricks on tuning your model.

In [3]:
YOUR_DIRECTORY = 'weedai_yolo'

!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}
%ls '/content/drive/MyDrive/' # should list everything in your Google Drive - double check that your project folder is there.

weedai_yolo/


**(first time only)**

Clone the YOLOv5 repository so we can use it to train our models. Only do this ONCE at the start of the project.

In [4]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}
!git clone https://github.com/ultralytics/yolov5 # clone the YOLOv5 repository. It is a large repository and may take some time depending on your internet speed.

/content/drive/MyDrive/weedai_yolo
Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.61 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (11690/11690), done.


# Downloading a Weed-AI dataset

For this example I've used the [Northern WA Wheatbelt Blue Lupins](https://weed-ai.sydney.edu.au/datasets/9df290f4-a29b-44b2-9de6-24bca1cee846) dataset but any of the other object detection datasets would work too, including the recrntly uploaded [Amsinckia in chickpeas](https://weed-ai.sydney.edu.au/datasets/21675efe-9d25-4096-be76-3a541475efd4) dataset.

Download the dataset to a default place on your computer and unzip it. Rename it to something more memorable, in this case `blue_lupins`. Then, we'll create a folder called `datasets` in the `yolov5` directory and move the Weed-AI download (now called `blue_lupins`) to that folder.

To summarise, the steps we will follow below are:
1. Download the dataset on Weed-AI by clicking the button 'Download in WEEDCOCO format'
2. Unzip the download and rename it to something memorable, in this case I've called it `blue_lupins`
3. Create the `datasets` folder in the `yolov5` directory using the code below
4. Move the Weed-AI download into the Google Drive `yolov5/datasets` folder. For me, this is now `'weedai_yolo/yolov5/datasets'`
5. Convert the data from WeedCOCO to [YOLOv5 format](https://roboflow.com/formats/yolov5-pytorch-txt)

Assuming you've downloaded the dataset, unzipped it and changed its name, I'll go through each of these other steps in more detail below.

In [5]:
YOUR_DATASET = 'blue_lupins' # this should match the memorable name of the Weed-AI download you just created.



Create the dataset folder where you'll move the unzipped folder renamed to `blue_lupins` to.

In [14]:
!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets

mkdir: cannot create directory ‘/content/drive/MyDrive/weedai_yolo/yolov5/datasets’: File exists


Once the dataset has downloaded and is in the datasets folder, it should have a similar structure to the following:
* yolov5/datasets
    * blue_lupins
        * images
        * weedcoco.json


# Convert weedCOCO to YOLO

The first step in the process is converting the downloaded weedCOCO dataset into the YOLO .txt format. The method below is adapted from the official [Ultralytics GitHub repository](https://github.com/ultralytics/JSON2YOLO/blob/master/labelbox_json2yolo.py). Don't worry too much about the code, though certainly check it out, just run the cell by pressing 'Play' on the left side.

In [15]:
import os
from pathlib import Path

import yaml
import shutil
from tqdm import tqdm
import contextlib
import json

import pandas as pd
import numpy as np
from PIL import Image
from collections import defaultdict

def make_dirs(dir='new_dir/'):
    # Create folders
    dir = Path(dir)
    for p in dir, dir / 'labels', dir / 'images':
        p.mkdir(parents=True, exist_ok=True)  # make dir
    return dir


def convert_weedcoco_json(json_dir=''):
    save_dir = make_dirs(dir=f'{json_dir}')  # output directory
    print()

    # Import json
    for json_file in sorted(Path(json_dir).resolve().glob('*.json')):
        fn = Path(save_dir) # / 'labels' # folder name
        fn.mkdir(exist_ok=True)
        with open(json_file) as f:
            data = json.load(f)

        # Create image dict
        images = {'%g' % x['id']: x for x in data['images']}
        # Create image-annotations dict
        imgToAnns = defaultdict(list)
        for ann in data['annotations']:
            imgToAnns[ann['image_id']].append(ann)


        # Write labels file
        for img_id, anns in tqdm(imgToAnns.items(), desc=f'Annotations {json_file}'):
            # print(img_id, anns)
            img = images['%g' % img_id]
            h, w, f = img['height'], img['width'], img['file_name']

            bboxes = []
            segments = []
            for ann in anns:
                # The COCO box format is [top left x, top left y, width, height]
                box = np.array(ann['bbox'], dtype=np.float64)
                box[:2] += box[2:] / 2  # xy top-left corner to center
                box[[0, 2]] /= w  # normalize x
                box[[1, 3]] /= h  # normalize y
                if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
                    continue

                cls = ann['category_id']  # class
                box = [cls] + box.tolist()
                if box not in bboxes:
                    bboxes.append(box)

            # Write
            with open((fn / f.replace('images', 'labels')).with_suffix('.txt'), 'a') as file:
                for i in range(len(bboxes)):
                    line = *(bboxes[i]),  # cls, box or segments
                    file.write(('%g ' * len(line)).rstrip() % line + '\n')

    # Save dataset.yaml
    names = [data['categories'][i]['name'].split(': ')[1] for i in range(len(data['categories']))]
    d = {'path': json_dir,
         'train': 'images/train',
         'val': 'images/train',
         'test': 'images/train',
         'nc': len(names),
         'names': names}  # dictionary

    with open(f"{save_dir}/weedcoco.yaml", 'w') as f:
        yaml.dump(d, f, sort_keys=False)


    print('\nweedCOCO to YOLO conversion completed successfully!')


In [17]:
WEED_COCO_LOCATION = f"/content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets/{YOUR_DATASET}"
#convert the weedcoco file
convert_weedcoco_json(json_dir=WEED_COCO_LOCATION)

Annotations /content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins/weedcoco.json: 100%|██████████| 217/217 [00:01<00:00, 133.83it/s]


weedCOCO to YOLO conversion completed successfully!


## Splitting the dataset into train/validation/test
An algorithm needs a training portion and a validation portion to check as it learns. The test portion is left entirely unseen and can be used later for more appropriate results and to make sure the algorithm hasn't overfit.

If you find the algorithm performs well on the training data but terribly on the val/test data, then it is likely overfitting. This is more common on small datasets and larger models when trained for many epochs.

In [18]:
from sklearn.model_selection import train_test_split

# Read images and annotations
images = [os.path.join(f'{WEED_COCO_LOCATION}/images', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/images')]
annotations = [os.path.join(f'{WEED_COCO_LOCATION}/labels', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-val-test splits 80-10-10%
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

%cd {WEED_COCO_LOCATION}
!mkdir images/train images/val images/test labels/train labels/val labels/test

/content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins


In [19]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')

In [20]:
# Check the images have been moved
print(len(os.listdir('images/train')), len(os.listdir('labels/train')))
print(len(os.listdir('images/val')), len(os.listdir('labels/val')))
print(len(os.listdir('images/test')), len(os.listdir('labels/test')))

173 173
22 22
22 22


# Preparing for training
Now we have all the splits made, we need to import some packages and install other YOLOv5 requirements before we can start training a model.

In [21]:
# import necessary packages
import torch
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(0)

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.5.0+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40, uuid=c628ab98-4025-867a-f9d9-32b1860d9a89, L2_cache_size=4MB)


In [24]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5
!pip install -r requirements.txt

/content/drive/MyDrive/weedai_yolo/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 26.6 MB/s eta 0:00:00


In [25]:
# Weights & Biases  (optional) - this will let you track and visualise the training process with a WandB account; however, it isn't necessary
%pip install -q wandb
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# YOLOv5 Training

Now we get to train a model! Change the name of your run to whatever you like, and try playing around with things like image size, batch size, epochs and YOLOv5 variant. Larger variants and larger images will probably do better, but require more memory. So if you run out of memory, just reduce image size or model variant size (choose M instead of X) and then try again.

Information on selecting batch size: https://twitter.com/rasbt/status/1617544195220312066


In [33]:
# train YOLOv5m
BATCH = 8
EPOCHS = 30
IMAGE_SIZE = 1280 # (should be one of 320, 640, 1280, 1920)
MODEL = 'm' # (should be one of 'n', 's', 'm', 'l', 'x' and must be in lower case)

# this is the name of your run, and how it will be saved
RUN_NAME = f'{YOUR_DATASET}_TRAIN_B{str(BATCH)}_E{str(EPOCHS)}_SZ{str(IMAGE_SIZE)}_M{MODEL}'

# avoid making any changes to the below, or check the Ultralytics docs for other commands
!python train.py --img {IMAGE_SIZE} --cfg yolov5{MODEL}.yaml --batch {BATCH} --epochs {EPOCHS} --data datasets/{YOUR_DATASET}/weedcoco.yaml --weights yolov5{MODEL}.pt --name {RUN_NAME}

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-10-26 22:15:18.873716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-26 22:15:18.893291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-26 22:15:18.899192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: ailaserweeder (ailaserweeder-university-at-buffalo). Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=yolov5m.yaml, data=datasets/blue_lupins/weed

# Detect
This is where you can run the model you've just trained on a sample video or other dataset to see how it goes. The --source flag below accepts videos, folders of images and images. All you need to do is upload these to the YOLOv5 datasets directory and then specify the name/path below.

In [40]:
DETECTION_FILES = 'blue_lupins/images/test/Blue_Lupins_Video_1.mp4' # e.g. 'test_video.mp4' OR test_image_directory OR test_image.jpg
CONFIDENCE_THRESHOLD = 0.50 # this should be between 0 and 1. It changes the cutoff value for a detection. Lower = more sensitive, higher = less sensitive

!python detect.py --source datasets/{DETECTION_FILES} --weights runs/train/{RUN_NAME}/weights/best.pt --name {RUN_NAME} --img {IMAGE_SIZE} --conf-thres 0.50

detect: weights=['runs/train/blue_lupins_TRAIN_B8_E30_SZ1280_Mm/weights/best.pt'], source=datasets/blue_lupins/images/test/Blue_Lupins_Video_1.mp4, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=blue_lupins_TRAIN_B8_E30_SZ1280_Mm, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/677) /content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins/images/test/Blue_Lupins_Video_1.mp4: 800x1280 3 lupinus cosentiniis, 66.4ms
video 1/1 (2/677) /content/drive/MyDrive/weedai_yolo/y